In [1]:
import sys
import numpy as np
import tensorflow as tf

sys.path.insert(0, "/home/anonymous/Desktop/eeg_sampling/modeling")
from dataio import readdata, readlabels, writedata

In [2]:
#Read training
train_dataset_right = readdata("./../curated/train_dataset_right")
train_dataset_left = readdata("./../curated/train_dataset_left")
train_dataset_avg = readdata("./../curated/train_dataset_avg")
train_labels = readlabels("./../curated/train_labels")

#Read test
test_dataset_right = readdata("./../curated/test_dataset_right")
test_dataset_left = readdata("./../curated/test_dataset_left")
test_dataset_avg = readdata("./../curated/test_dataset_avg")
test_labels = readlabels("./../curated/test_labels")

print('Training:', train_dataset_right.shape, train_dataset_left.shape, train_dataset_avg.shape, train_labels.shape)
print('Testing:', test_dataset_right.shape, test_dataset_left.shape, test_dataset_avg.shape, test_labels.shape)

Training: (451, 16, 480) (451, 16, 480) (451, 16, 480) (451,)
Testing: (115, 16, 480) (115, 16, 480) (115, 16, 480) (115,)


In [3]:
n_input = 16*480 # EEG data input (16 * 480 image)
n_classes = 3 # EEG total classes ("nothing", "up", "down", "left", "right")

In [4]:
def onehot(data, n_classes):
    b = np.zeros((data.size, n_classes))
    b[np.arange(data.size), data] = 1
    return(b)

train_labels = onehot(train_labels, n_classes)
test_labels = onehot(test_labels, n_classes)

In [29]:
train_dataset_right.shape = (train_dataset_right.shape[0], n_input)
test_dataset_right.shape = (test_dataset_right.shape[0], n_input)

In [30]:
print(train_dataset_right.shape)

(451, 7680)


In [27]:
def next_batch(train_dataset_right, batch):
    rand = np.random.random_integers(0,train_dataset_right.shape[0]-1,size=batch)
    return(train_dataset_right[rand], train_labels[rand])

In [25]:
sess = tf.InteractiveSession()

In [11]:
# Create the modelten
x = tf.placeholder(tf.float32, [None, n_input])
W = tf.Variable(tf.zeros([n_input, n_classes]))
b = tf.Variable(tf.zeros([n_classes]))
y = tf.nn.softmax(tf.matmul(x, W) + b)

In [19]:
# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, n_classes])
cross_entropy = -tf.reduce_sum(y_ * tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

In [ ]:
data = train_dataset_right

In [ ]:
batch_size = 150
# Train
tf.initialize_all_variables().run()
for i in range(2000):
  batch_xs, batch_ys = next_batch(data, batch_size)
  train_step.run({x: batch_xs, y_: batch_ys})

In [ ]:
# Test trained model
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval({x: test_dataset_right, y_: test_labels}))

In [37]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [38]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [39]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

In [40]:
x_image = tf.reshape(x, [-1,480,16,1])

In [41]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [42]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [51]:
W_fc1 = weight_variable([15 * 16 * 128, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 15*16*128])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [52]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [53]:
W_fc2 = weight_variable([1024, n_classes])
b_fc2 = bias_variable([n_classes])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [54]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [60]:
sess.run(tf.initialize_all_variables())
for i in range(1000):
  batch = next_batch(train_dataset_right, 20)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: test_dataset_right, y_: test_labels, keep_prob: 1.0}))

step 0, training accuracy 0.3
step 100, training accuracy 0.35
step 200, training accuracy 0.45
step 300, training accuracy 0.6
step 400, training accuracy 0.35
step 500, training accuracy 0.5
step 600, training accuracy 0.45
step 700, training accuracy 0.75
step 800, training accuracy 0.55
step 900, training accuracy 0.8
test accuracy 0.33913
